# This script is daily BMO Transactions update for Shay

In [1]:
import os
import pandas as pd
import re

In [2]:
#specify the folder path where the source files are located
file_path = r'C:\Users\WRFI\OneDrive - Western Restaurant Franchise Inc\Documents - WRFIFiles\WRFI ACCOUNTING\E.  BANK & SALES\2024\A. BMO\P2\WRFI P2 BMO transactions.xlsx'
sheet_name = 'Sheet1'

In [3]:
df = pd.read_excel(file_path, sheet_name=sheet_name)

In [4]:
#specify the folder path where the transaction mapping files are located
mapping_data = r'C:\Users\WRFI\OneDrive - Western Restaurant Franchise Inc\Documents - WRFIFiles\WRFI ACCOUNTING\E.  BANK & SALES\2023\A. BMO\BMO transactions_mapping.xlsx'

In [5]:
mapping_df = pd.read_excel(mapping_data)

In [6]:
df.columns

Index([             'Processing Date',            'Processing Date.1',
            'Transaction Description',                     'Cust Ref',
                           'Bank Ref',            -2543127.4300000006,
                    3135412.569999998,                 'Credit/Debit',
                'Transaction Details',                   'Entry Type',
                             'Respo.',                       'Posted',
                            'Recon. ', 'Txn details brginning 4 char'],
      dtype='object')

In [7]:
mapping_df.columns

Index(['Transaction Description', 'Full Details', 'Details (start with)',
       'Details (ends with)', 'Category', 'Notes', 'Unnamed: 6',
       'Unsure Category'],
      dtype='object')

In [8]:
def apply_logic(df, mapping_df):
    if df['Transaction Description'] in ['Merchant Deposit  / Adjustment', 'Merchant Deposit / Adjustment']:
        return "Mobile"
    elif df['Transaction Description'] in ['FCCS Deposit / Correction', 'Credit Memo']:
        return "Cash Deposits"
    elif df['Transaction Description'] == "Cheque":
        return "Cheque beanworks"
    elif df['Transaction Description'] == "Service Charge":
        return "Fee"
    elif df['Transaction Description'] == "Interest / Adjustment":
        return "Loan"
    elif not df['Transaction Details'] or df['Transaction Details'] not in mapping_df['Full Details'].tolist():
        left_six_J = df['Transaction Details'][:6]
        right_six_J = df['Transaction Details'][-24:]
        matching_rows_left = mapping_df[(mapping_df['Transaction Description'] == df['Transaction Description']) & (mapping_df['Details (start with)'].str[:6] == left_six_J)]
        if not matching_rows_left.empty:
            return matching_rows_left.iloc[0]['Category']
        matching_rows_right = mapping_df[(mapping_df['Transaction Description'] == df['Transaction Description']) & (mapping_df['Details (start with)'].str[-24:] == right_six_J)]
        if not matching_rows_right.empty:
            return matching_rows_right.iloc[0]['Category']            
    elif not df['Transaction Details'] or df['Transaction Details'] in mapping_df['Full Details'].tolist():
        matching_rows = mapping_df[(mapping_df['Transaction Description'] == df['Transaction Description']) & (mapping_df['Full Details'] == df['Transaction Details'])]
        if not matching_rows.empty:
            return matching_rows.iloc[0]['Category']
    return ''

In [9]:
df['Entry Type'] = df.apply(lambda row: apply_logic(row, mapping_df), axis=1)

In [10]:
def apply_name(df):
    if df['Entry Type'] == "Accounts payable":
        return "Shauna"
    else:
        return "Shay"
    return ''

In [11]:
df['Respo.'] = df.apply(lambda row: apply_name(row), axis=1)

In [12]:
# Write the DataFrame to an Excel file
excel_output_path = r'C:\Users\WRFI\OneDrive - Western Restaurant Franchise Inc\Documents - WRFIFiles\WRFI ACCOUNTING\E.  BANK & SALES\2024\A. BMO\P2\WRFI P2 BMO transactions - coded with python.xlsx'
df.to_excel(excel_output_path, index=False)